# Task for Today  

***

## Financial News Sentiment Prediction  

Given *financial news headlines*, let's try to predict the **sentiment** of a given headline.

We will use a TensorFlow RNN to make our predictions.

# Getting Started

In [1]:
# import libraries
# numpy and pandas for working with data
import numpy as np
import pandas as pd

# these libraries for preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# for model
import tensorflow as tf

# for test
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
# input the data from dataset
data = pd.read_csv('../input/sentiment-analysis-for-financial-news/all-data.csv', names=['Label', 'Text'], encoding='latin-1')

In [3]:
# visualize data
data

,Label,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   4846 non-null   object
 1   Text    4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


# Preprocessing

In [5]:
# encode each word as it's own integer + sort words in order of frequency
def get_sequences(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    
    sequences = tokenizer.texts_to_sequences(texts)
    print("Vocab length:", len(tokenizer.word_index) + 1)
    
    max_seq_length = np.max(list(map(lambda x: len(x), sequences)))
    print("Maximum sequence length:", max_seq_length)
    
    sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')
    # padding='post' means 0 go to the end
    
    return sequences

In [6]:
# makes a copy of what we have
def preprocess_inputs(df):
    df = df.copy()
    
    sequences = get_sequences(df['Text'])
    
    label_mapping = {
        'negative': 0,
        'neutral': 1,
        'positive': 2
    }

    # performs the mapping function
    y = df['Label'].replace(label_mapping)
    
    train_sequences, test_sequences, y_train, y_test = train_test_split(sequences, y, train_size=0.7, shuffle=True, random_state=1)
    
    return train_sequences, test_sequences, y_train, y_test

In [7]:
train_sequences, test_sequences, y_train, y_test = preprocess_inputs(data)

Vocab length: 10123
Maximum sequence length: 71


In [8]:
# vector length is 71
train_sequences

array([[5442,  510,   16, ...,    0,    0,    0],
       [  22, 1628,    4, ...,    0,    0,    0],
       [1141,  936,  136, ...,    0,    0,    0],
       ...,
       [   1,  419,   16, ...,    0,    0,    0],
       [2586,  123, 3247, ...,    0,    0,    0],
       [  30,  615,  555, ...,    0,    0,    0]], dtype=int32)

In [9]:
# numeric integer labels
y_train

545     2
2374    0
4217    1
1071    1
716     2
       ..
2895    1
2763    1
905     2
3980    1
235     2
Name: Label, Length: 3392, dtype: int64

# Training

In [10]:
inputs = tf.keras.Input(shape=(train_sequences.shape[1],))
x = tf.keras.layers.Embedding(
    input_dim=10123,
    output_dim=128,
    input_length=train_sequences.shape[1]
)(inputs)

# Using GRU model which is type of RNN
x = tf.keras.layers.GRU(256, return_sequences=True, activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_sequences,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
85/85 [==============================] - 12s 124ms/step - loss: 0.8864 - accuracy: 0.6204 - val_loss: 0.7976 - val_accuracy: 0.6333
Epoch 2/100
85/85 [==============================] - 10s 118ms/step - loss: 0.5147 - accuracy: 0.7866 - val_loss: 0.7009 - val_accuracy: 0.7084
Epoch 3/100
85/85 [==============================] - 10s 121ms/step - loss: 0.1627 - accuracy: 0.9553 - val_loss: 1.1588 - val_accuracy: 0.6907
Epoch 4/100
85/85 [==============================] - 10s 118ms/step - loss: 0.0428 - accuracy: 0.9869 - val_loss: 1.0043 - val_accuracy: 0.6981
Epoch 5/100
85/85 [==============================] - 10s 117ms/step - loss: 0.0143 - accuracy: 0.9968 - val_loss: 1.0801 - val_accuracy: 0.7040


# Results

In [11]:
results = model.evaluate(test_sequences, y_test, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.63234
Test Accuracy: 74.42%


In [12]:
# see how many pos/neg/neu
y_test.value_counts()

1    850
2    420
0    184
Name: Label, dtype: int64